In [4]:
import os
import subprocess
import glob



In [8]:
# TOOLS PATH
tools_path = "/media/datastorage/it_cast/omnis_microservice_db/tools/thermofisher/"
mzml_dir = "/media/datastorage/it_cast/metabolomica/DDA_File/"

raw_files = glob.glob(os.path.join(mzml_dir, "*.raw"))
print(raw_files)

['/media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_03.raw', '/media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_01.raw', '/media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_02.raw']


In [11]:
# thermo file parser
thermo_file_parser = os.path.join(tools_path, "ThermoRawFileParser.exe")

for raw_file in raw_files:

    cmd = [
        "mono", thermo_file_parser, "-i", raw_file, "-o", mzml_dir, "-f", "1"
    ]
    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing file {raw_file}: {e}")

2025-10-02 11:24:03 INFO Started parsing /media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_03.raw
2025-10-02 11:24:06 INFO Processing 11217 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2025-10-02 11:24:59 INFO Finished parsing /media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_03.raw
2025-10-02 11:24:59 INFO Processing completed 0 errors, 0 warnings
2025-10-02 11:24:59 INFO Started parsing /media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_01.raw
2025-10-02 11:25:03 INFO Processing 11306 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2025-10-02 11:25:51 INFO Finished parsing /media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_01.raw
2025-10-02 11:25:51 INFO Processing completed 0 errors, 0 warnings
2025-10-02 11:25:51 INFO Started parsing /media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_02.raw
2025-10-02 11:25:55 INFO Processing 11334 MS scans
10% 20% 30

In [12]:
mzml_files = glob.glob(os.path.join(mzml_dir, "*.mzML"))
print(mzml_files)

['/media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_02.mzML', '/media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_03.mzML', '/media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_01.mzML']


In [16]:
# create the results directory if it doesn't exist
results_dir = "/media/datastorage/it_cast/metabolomica/DDA_File/results/"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)


for file_index in range(len(mzml_files)):
    # define the output name
    output_file = os.path.join(results_dir, f"peak_file_{file_index + 1}.mzML")
    cmd = [
        "PeakPickerHiRes",
        "-in", mzml_files[file_index],
        "-out", output_file,
    ]
    subprocess.run(cmd)
    print(f"Processed file {file_index + 1} of {len(mzml_files)}")

Progress of 'loading spectra list':
-- done [took 2.46 s (CPU), 3.05 s (Wall)] -- 
Progress of 'loading chromatogram list':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Progress of 'picking peaks':
-- done [took 0.06 s (CPU), 0.07 s (Wall)] -- 
Progress of 'storing mzML file':
-- done [took 0.42 s (CPU), 0.44 s (Wall)] -- 
Picked spectra by MS-level:
  MS-level 1: 0 / 2693
  MS-level 2: 0 / 8641
PeakPickerHiRes took 4.02 s (wall), 3.13 s (CPU), 0.24 s (system), 2.89 s (user); Peak Memory Usage: 236 MB.
Processed file 1 of 3
Progress of 'loading spectra list':
-- done [took 2.49 s (CPU), 2.81 s (Wall)] -- 
Progress of 'loading chromatogram list':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Progress of 'picking peaks':
-- done [took 0.05 s (CPU), 0.06 s (Wall)] -- 
Progress of 'storing mzML file':
-- done [took 0.42 s (CPU), 0.44 s (Wall)] -- 
Picked spectra by MS-level:
  MS-level 1: 0 / 2670
  MS-level 2: 0 / 8547
PeakPickerHiRes took 3.70 s (wall), 3.13 s (CPU), 0.22 s (system), 

In [17]:
peak_files = glob.glob(os.path.join(results_dir, "peak_file_*.mzML"))


# feature detection algorithm
for file_index in range(len(peak_files)):
    # define the output name
    output_file = os.path.join(results_dir, f"feature_file_{file_index + 1}.featureXML")
    cmd = [
        "FeatureFinderMetabo",
        "-in", peak_files[file_index],
        "-out", output_file,
        "-ini", "/media/datastorage/it_cast/omnis_microservice_db/metabolomics/article_analysis/file.ini"
    ]
    subprocess.run(cmd)
    print(f"Processed feature file {file_index + 1} of {len(peak_files)}")

Progress of 'loading spectra list':
-- done [took 0.91 s (CPU), 1.15 s (Wall)] -- 
Progress of 'loading chromatogram list':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Progress of 'mass trace detection':
-- done [took 14.46 s (CPU), 15.42 s (Wall)] -- 
Progress of 'elution peak detection':
-- done [took 2.23 s (CPU), 2.48 s (Wall)] -- 
Progress of 'assembling mass traces to features':
Loading metabolite isotope model with 5% RMS error
-- done [took 0.51 s (CPU), 0.54 s (Wall)] -- 
-- FF-Metabo stats --
Input traces:    11392
Output features: 10758 (total trace count: 11392)
Progress of 'Storing featureXML file':
-- done [took 0.10 s (CPU), 0.10 s (Wall)] -- 
FeatureFinderMetabo took 28.71 s (wall), 25.67 s (CPU), 0.34 s (system), 25.33 s (user); Peak Memory Usage: 612 MB.
Processed feature file 1 of 3
Progress of 'loading spectra list':
-- done [took 1.00 s (CPU), 1.24 s (Wall)] -- 
Progress of 'loading chromatogram list':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Progress of '

In [19]:
# Map aligner   
feature_files = glob.glob(os.path.join(results_dir, "feature_file_*.featureXML"))
# the aligned outputs should be of the same number of input files
aligned_files = [os.path.join(results_dir, f"aligned_{i + 1}.featureXML") for i in range(len(feature_files))]

cmd = [
    "MapAlignerPoseClustering",
    "-in", *feature_files,
    "-out", *aligned_files,
    "-reference:file", feature_files[0]
]


try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during map alignment: {e}")


Progress of 'Aligning input maps':
-- done [took 1.43 s (CPU), 1.74 s (Wall)] -- 
MapAlignerPoseClustering took 2.14 s (wall), 1.79 s (CPU), 0.02 s (system), 1.77 s (user); Peak Memory Usage: 64 MB.


In [20]:
consensus_file = os.path.join(results_dir, "consensus.consensusXML")
cmd = [
    "FeatureLinkerUnlabeledQT",
    "-in", *aligned_files,
    "-out", consensus_file
]
try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during feature linking: {e}")

Linking 3 featureXMLs.
Progress of 'reading input':
-- done [took 1.00 s (CPU), 1.09 s (Wall)] -- 
Progress of 'Linking features':
-- done [took 0.20 s (CPU), 0.22 s (Wall)] -- 
Number of consensus features:
  of size  3:   4923
  of size  2:   4736
  of size  1:   9525
  total:       19184
FeatureLinkerUnlabeledQT took 1.51 s (wall), 1.37 s (CPU), 0.03 s (system), 1.34 s (user); Peak Memory Usage: 64 MB.


In [29]:
# Accurate Mass Search
out_annotation = os.path.join(results_dir, "annotated_aligned_1.featureXML")
cmd = [
    "AccurateMassSearch",
    "-in", aligned_files[0],
    "-out", os.path.join(results_dir, "annotated_1.mzTab"),
    "-out_annotation",out_annotation  
]

try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during accurate mass search: {e}")

Read 11531 entries from mapping file!
Read 33 entries from adduct file '/usr/share/openms/CHEMISTRY/PositiveAdducts.tsv'.
Read 15 entries from adduct file '/usr/share/openms/CHEMISTRY/NegativeAdducts.tsv'.

Found 8845 matched masses (with at least one hit each)
from 10758 features
  --> 82% explained
Hits by adduct: #peaks explained (# matching db entries)'
  '2M+3H2O+2H;2+' : 1928 (11289)
  '2M+CH3CN+H;1+' : 807 (3759)
  '2M+CH3CN+Na;1+' : 624 (2915)
  '2M+H;1+' : 947 (4671)
  '2M+K;1+' : 536 (2072)
  '2M+NH4;1+' : 891 (4460)
  '2M+Na;1+' : 747 (4195)
  'M+2CH3CN+2H;2+' : 1266 (8129)
  'M+2CH3CN+H;1+' : 1463 (7117)
  'M+2H+Na;3+' : 587 (3913)
  'M+2H;2+' : 1125 (15508)
  'M+2K-H;1+' : 658 (2303)
  'M+2Na-H;1+' : 1056 (6631)
  'M+2Na;2+' : 995 (8188)
  'M+3CH3CN+2H;2+' : 864 (6540)
  'M+3H;3+' : 568 (4112)
  'M+3Na;3+' : 501 (5453)
  'M+C2H6OS;1+' : 561 (1950)
  'M+C3H8O+Na+H;1+' : 1321 (5378)
  'M+C3H8O;1+' : 922 (2780)
  'M+CH3CN+2H;2+' : 668 (2885)
  'M+CH3CN+H;1+' : 1933 (14561)
  

In [30]:
# assay library generation
cmd = [
    "AssayGeneratorMetabo",
    "-in", mzml_files[0],
    "-in_id", out_annotation,
    "-out", os.path.join(results_dir, "assay_library.tsv"),
]
try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during assay library generation: {e}")


Input MzML: 
 /media/datastorage/it_cast/metabolomica/DDA_File/20250522_TCAM_POS_Acetate_02.mzML
Input FeatureXML: 
 /media/datastorage/it_cast/metabolomica/DDA_File/results/annotated_aligned_1.featureXML
Original paths: 
 /media/datastorage/it_cast/metabolomica/DDA_File/results/peak_file_3.mzML
AssayGeneratorMetabo took 5.40 s (wall), 5.08 s (CPU), 0.14 s (system), 4.94 s (user); Peak Memory Usage: 315 MB.
